Potential: event types (pass, duel, etc.), most common event (max frequency).

The players are important for the transitions. How do we encode them? Distribution, counts?

Distribution of positions (GK, DEF, MID, ATT). Prune events for only PASSES, SHOTS, DUEL (TACKLES?)

In [ ]:
df_players = pd.read_json("data/players.json")
df_players = pd.json_normalize(df_players.to_dict("records"), sep="_")

# replace null strings with actual nulls
df_players = df_players.replace("null", pd.NA)

# decode unicode escape sequences in name columns
for col in ["firstName", "middleName", "lastName", "shortName"]:
    df_players[col] = df_players[col].apply(
        lambda x: x.encode().decode("unicode-escape") if isinstance(x, str) else x
    )

df_players.rename(
    columns={"wyId": "player_id", "currentTeamId": "team_id"}, inplace=True
)

# rename all cols to snake_case
df_players.columns = [
    "".join(["_" + i.lower() if i.isupper() else i for i in col]).lstrip("_")
    for col in df_players.columns
]

df_players.set_index("player_id", inplace=True)
df_players

In [ ]:
match_id = 8658  # France vs Croatia World Cup Final 2018

data = statsbomb.load_open_data(match_id)
data = data.transform(
    Orientation.ACTION_EXECUTING_TEAM
)  # the team that executes the action always plays from left to right (with x=100 being the opponent's goal)

home_team, away_team = data.metadata.teams
home_team_id, away_team_id = home_team.team_id, away_team.team_id
print(f"Home team: {home_team.name} ({home_team_id})")
print(f"Away team: {away_team.name} ({away_team_id})")

df_raw = data.to_df(*zt.INCLUDE_COLS, engine="pandas")
df_raw.set_index("event_id", inplace=True)
df_raw = zt.transform(df_raw)
df_raw